In [1]:
import pandas as pd

In [71]:
df1 = pd.read_excel("Training.xlsx")
df2 = pd.read_excel("Validation.xlsx")

In [72]:
df1.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)
df2.drop(["연령", "성별", "신체질환", "상황키워드", "감정_소분류",  "시스템문장1", "시스템문장2", "시스템문장3"], axis=1, inplace=True)

In [79]:
df1.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df1.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)

df2.rename(columns={"사람문장1" : "sentence"}, inplace=True)
df2.rename(columns={'감정_대분류': 'Emotion'}, inplace=True)


In [80]:
df_train = pd.concat([
    df1[['Emotion', 'sentence']],
    df1[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df1[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)

df_valid = pd.concat([
    df2[['Emotion', 'sentence']],
    df2[['Emotion', '사람문장2']].rename(columns={'사람문장2': 'sentence'}),
    df2[['Emotion', '사람문장3']].rename(columns={'사람문장3': 'sentence'})
], ignore_index=True)


In [84]:
df_train = df_train.dropna()
df_valid = df_valid.dropna()

In [88]:
df_train.to_csv("training_data.csv", index=False)
df_valid.to_csv("validation_data.csv", index=False)